In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [92]:
train_data = pd.read_csv('C:/Users/cat_w/Documents/Second Year/QMIND/Titanic data/train.csv')

In [93]:
test_data = pd.read_csv("C:/Users/cat_w/Documents/Second Year/QMIND/Titanic data/test.csv")

In [94]:
train_data = pd.read_csv('C:/Users/cat_w/Documents/Second Year/QMIND/Titanic data/train.csv')
test_data = pd.read_csv("C:/Users/cat_w/Documents/Second Year/QMIND/Titanic data/test.csv")
all_data = [train_data, test_data]

#FEATURE ENGINEERING??
for dataset in all_data:
    
    #Name
    dataset["NameLength"] = dataset["Name"].apply(len)
    
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir',
                                                 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    dataset['Title'] = dataset['Title'].fillna(0)
    
    #Sex
    dataset["Sex"] = dataset["Sex"].map( {"female": 1, "male": 0} ).astype(int)
    
    #Cabin
    dataset['HasCabin'] = dataset["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
                        
    #fill in missing fare values
    dataset["Fare"].fillna(dataset["Fare"].median(), inplace = True)
    #divide data by quantiles
    quarter = dataset["Fare"].quantile(.25)
    half = dataset["Fare"].quantile(.5)
    three_quarters = dataset["Fare"].quantile(.75)
    #map fare
    dataset.loc[dataset["Fare"] <= quarter, "Fare"] = 0
    dataset.loc[(dataset["Fare"] > quarter) & (dataset["Fare"] <= half), "Fare"] = 1
    dataset.loc[(dataset["Fare"] > half) & (dataset["Fare"] <= three_quarters), "Fare"] = 2
    dataset.loc[dataset["Fare"] > three_quarters, "Fare"] = 3
    dataset["Fare"] = dataset["Fare"].astype(int)
    
    #fill missing ages with the median then categorize and map
    dataset["Age"].fillna(dataset["Age"].median(), inplace = True)
    #divide data by quantiles
    first = dataset["Age"].quantile(.2)
    second = dataset["Age"].quantile(.4)
    third = dataset["Age"].quantile(.6)
    fourth = dataset["Age"].quantile(.8)
    #map age 
    dataset.loc[ dataset["Age"] <= first, "Age"] = 0
    dataset.loc[(dataset["Age"] > first) & (dataset["Age"] <= second), "Age"] = 1
    dataset.loc[(dataset["Age"] > second) & (dataset["Age"] <= third), "Age"] = 2
    dataset.loc[(dataset["Age"] > third) & (dataset["Age"] <= fourth), "Age"] = 3
    dataset.loc[ dataset["Age"] > fourth, "Age"] = 4 
 
    #get family sizes based on SibSp and Parch
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1

    #fill missing values for embarked and map
    dataset["Embarked"] = dataset["Embarked"].fillna('S')
    dataset["Embarked"] = dataset["Embarked"].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
   

In [95]:
#drop unused features
train_data = train_data.drop(["Name", "PassengerId", "Ticket", "Cabin"], axis = 1)
test_data = test_data.drop(['Name', "PassengerId", "Ticket", "Cabin"], axis = 1)


In [96]:
import matplotlib.pyplot as plt
rs = np.random.RandomState(0)

#make correlation matrix
corr = train_data.corr()
corr.style.background_gradient(cmap='coolwarm')


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,NameLength,Title,HasCabin,FamilySize
Survived,1,-0.338481,0.543351,-0.00228739,-0.0353225,0.0816294,0.299357,0.106811,0.33235,0.393241,0.316912,0.016639
Pclass,-0.338481,1,-0.1319,-0.336503,0.0830814,0.0184427,-0.634271,0.0457018,-0.220001,-0.160323,-0.725541,0.0659969
Sex,0.543351,-0.1319,1,-0.03585,0.114631,0.245489,0.243613,0.116569,0.448759,0.486836,0.140391,0.200988
Age,-0.00228739,-0.336503,-0.03585,1,-0.177746,-0.0691852,0.151904,-0.0778803,0.0889087,0.00794962,0.246223,-0.156047
SibSp,-0.0353225,0.0830814,0.114631,-0.177746,1,0.414838,0.393025,-0.0599614,0.165019,0.272835,-0.04046,0.890712
Parch,0.0816294,0.0184427,0.245489,-0.0691852,0.414838,1,0.393881,-0.0786647,0.252282,0.318745,0.036987,0.783111
Fare,0.299357,-0.634271,0.243613,0.151904,0.393025,0.393881,1,-0.0981614,0.335339,0.328776,0.500936,0.465396
Embarked,0.106811,0.0457018,0.116569,-0.0778803,-0.0599614,-0.0786647,-0.0981614,1,-0.107749,0.0387645,0.0137744,-0.0802812
NameLength,0.33235,-0.220001,0.448759,0.0889087,0.165019,0.252282,0.335339,-0.107749,1,0.473155,0.190943,0.23882
Title,0.393241,-0.160323,0.486836,0.00794962,0.272835,0.318745,0.328776,0.0387645,0.473155,1,0.119939,0.345713


In [89]:
#Choose features based on correlation matrix: looking for "most red" boxes in relation to "Survived"
#Seems like SibSp and Parch can be dropped due to strong correlation with familySize

#ensemble learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from statistics import mode
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, VotingClassifier)

features = ["Sex", "Title", "Fare","NameLength", "FamilySize", "HasCabin", "Embarked"]
x_train = pd.get_dummies(train_data[features])
y_train = train_data["Survived"]
x_test = pd.get_dummies(test_data[features])

#generate a few models and train
model1 = GradientBoostingClassifier(n_estimators = 500)
model2 = RandomForestClassifier(n_estimators = 500, max_depth=5)
model3 = AdaBoostClassifier(n_estimators = 500)
model4 = GaussianProcessClassifier()
model5 = DecisionTreeClassifier()

model1.fit(x_train, y_train)
model2.fit(x_train, y_train)
model3.fit(x_train, y_train)
model4.fit(x_train, y_train)
model5.fit(x_train, y_train)

#voting classifier
voting_classifier = VotingClassifier(estimators=[
    ('gradient_boosting',  model1),
    ('random_forest', model2),
    ('adaBoost_classifier', model3),
    ('gaussian_process_classifier', model4), 
    ('decision_tree', model5)
],voting='hard')

voting = voting_classifier.fit(x_train,y_train)
vote_pred = voting.predict(x_test)
print(vote_pred)

#Generate Submission File 
PassengerId = test_data["PassengerId"]
submission = pd.DataFrame({ 'PassengerId': PassengerId,'Survived': vote_pred })
submission.to_csv("my_submission3.csv", index=False)


[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1
 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0
 0 1 1 1 1 0 0 1 0 0 1]


In [90]:
#cross fold validation
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, KFold
from matplotlib import pyplot as plt

train1, test1, train2, test2= train_test_split(x_train, y_train)

scores = []
folds = KFold(n_splits=10)  #make 10 folds

for train_index, test_index in folds.split(x_train):

    train1, test1, train2, test2 = x_train.iloc[train_index], x_train.iloc[test_index], y_train[train_index], y_train[test_index]
    voting_classifier.fit(train1, train2)
    scores.append(voting_classifier.score(test1, test2))
    
print(np.mean(scores))

0.8271660424469414
